In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from azure.eventgrid import EventGridClient
from msrest.authentication import TopicCredentials
import uuid
import json

In [2]:
kv_scope = 'key-vault-secret'

# Variables
eventgrid_accesskey = dbutils.secrets.get(scope=kv_scope, key='traffic-eventgrid-accesskey') 
eventgrid_topic = dbutils.secrets.get(scope=kv_scope, key='traffic-eventgrid-topicendpoint') 

In [3]:
class EventGrid_ForeachWriter:
  def __init__(self, topic, key):
    self.eg_accesskey = key
    self.eg_topicurl = topic
    
  def open(self, partition_id, epoch_id):
    # This is called first when preparing to send multiple rows.
    credentials = TopicCredentials(self.eg_accesskey)
    self.event_grid_client = EventGridClient(credentials)
    return True

  def process(self, row):
    # This is called for each row after open() has been called.
    print(row)
    self.publish_event(row)

  def close(self, err):
    # This is called after all the rows have been processed.
    if err:
      raise err
      
  def publish_event(self, row):
    print(row['LicensePlate'])
    
    subject = 'traffic/{}/{}'.format(
      row['TrajectId'],
      row['LicensePlate'])
    message_body = json.dumps(row.asDict())
    self.event_grid_client.publish_events(
        self.eg_topicurl,
        events=[{
            'id' : str(uuid.uuid4()),
            'subject' : subject,
            'data': message_body,
            'event_type': 'SuspectedCarDetected',
            'event_time': datetime.utcnow(),
            'data_version': 1
        }]
    )

In [4]:
# We leave out the timestamp, because that would result in a json serialization exception
cameraStream = spark.readStream.format("delta").table("cameratelemetry").select('TrajectId', 'CameraId', 'Lane', 'Country', 'LicensePlate', 'Make', 'Color')

In [5]:
# Open stream to delta table suspected_cars - loaded as a stream
suspectedCarStream = spark.readStream.format("delta").table("SuspectedCars")

In [6]:
# Join delta table with incoming telemetry stream
detectedCars = cameraStream.join(suspectedCarStream, "LicensePlate")
query = detectedCars.writeStream.foreach(EventGrid_ForeachWriter(eventgrid_topic, eventgrid_accesskey)).start()